In [2]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

gc.enable()

Using TensorFlow backend.


In [3]:
spectrums_index = np.array(glob.glob('E:/note_detection/spectrogram/*'))
labels_index = np.array(glob.glob('E:/note_detection/label/*'))

count = 0
for i in range(len(labels_index)):
    num = np.load(spectrums_index[i]).shape[1]//np.load(labels_index[i]).shape[0]
    if num == 24:
        count += 1
    elif num != 23:
        print(num)

In [4]:
num_samples = 5000
height = 108
width = 108
steps = 20
spec_list = spectrums_index[:num_samples]
labe_list = labels_index[:num_samples]

In [5]:
kf = KFold(n_splits=12, random_state=7)

In [6]:
def generate_random_cnn_lstm_input_and_target_label(spectrogram, label):
    length = np.shape(spectrogram)[1] - 108
    random_point = np.random.randint(0,length)

    cnn_input = spectrogram[:,random_point:random_point+108]
    lstm_input = label[random_point//27:random_point//27+20,:]
    target_label = label[random_point//27+20,:]
        
    return cnn_input, lstm_input, target_label

In [7]:
def classifier_1(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = BatchNormalization()(cnn_inputs)
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [8]:
def classifier_3(optimizer='adam', loss='binary_crossentropy', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(64, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([lstm_layers, layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[lstm_inputs, cnn_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [9]:
train_order = []
test_order = []
length = np.arange(num_samples)

for x, y in kf.split(range(num_samples)):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [10]:
def inputs_to_model(spectrum_list, labels_list, order, num, start, ending):
    lstm_output = []
    cnn_output = []
    y_train = []
    
    import time
    start_ = time.time()

    for file in order[num][start:ending]:
        spectrum = np.load(spectrum_list[file])
        labels = np.load(labels_list[file])
        gap = spectrum.shape[1] // labels.shape[0]
        
        _lstm = np.zeros((20, 24))
        for count, i in enumerate(labels):
            _cnn = spectrum[:, gap*count:108+gap*count]
            if _cnn.shape == (height, width):
                lstm_output.append(_lstm)
                cnn_output.append(_cnn.reshape((height, width, 1)))
                y_train.append(i)
        
            _lstm = _lstm[1:, :]
            _lstm = np.append(_lstm, i.reshape((1, 24)), axis=0)
        
    lstm_inputs = np.array(lstm_output)
    cnn_inputs = np.array(cnn_output)
    labels_inputs = np.array(y_train)
    print(time.time()-start_)
    return lstm_inputs, cnn_inputs, labels_inputs

In [34]:
def generate_database(glob_spectrogram, glob_label, times=10):
    cnn_inputs = []
    lstm_inputs = []
    labels_inputs = []
    for spectrogram_,label_ in zip(glob_spectrogram,glob_label):
        spectrogram = np.load(spectrogram_)
        label = np.load(label_)
        for i in range(times):
            cnn_input, lstm_input, target_label = generate_random_cnn_lstm_input_and_target_label(spectrogram, label)
            cnn_inputs.append(cnn_input)
            lstm_inputs.append(lstm_input)
            labels_inputs.append(target_label)
    return np.array(cnn_inputs), np.array(lstm_inputs), np.array(labels_inputs)

In [35]:
def inputs_to_model_split(spectrum_list, labels_list, order, num, start, ending, times=10):
    cnn_inputs, lstm_inputs, labels_inputs = generate_database(spectrum_list[order[num][start:ending]], 
                                                               labels_list[order[num][start:ending]], times)
    cnn_inputs = cnn_inputs.reshape((cnn_inputs.shape[0], 108, 108, 1))
    return lstm_inputs, cnn_inputs, labels_inputs

In [36]:
lstm_test, cnn_test, labels_test = inputs_to_model_split(spec_list, labe_list, test_order, 0, 0, test_order[0].shape[0]+1, times=20)

In [37]:
lstm_test.shape

(8340, 20, 24)

In [38]:
cnn_test.shape

(8340, 108, 108, 1)

In [39]:
labels_test.shape

(8340, 24)

In [41]:
def train_test_whole(classifier, num, name, spectrum_list, labels_list, epochs=100, batchsize=32, times=10):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model_split(spectrum_list, labels_list, train_order, 
                                                             num, 0, train_order[num].shape[0]+1, times)

    classifier.fit([lstm_inputs, cnn_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([lstm_test, cnn_test], labels_test))
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [23]:
def train_test_part(classifier, num, name, spectrum_list, labels_list, epochs=100, batchsize=32, split=7):
    
    num_samples = train_order[num].shape[0]
    
    for j in range(epochs):
        print(
           """
        
        THIS IS EPOCH_{}""".format(j))
        for i in range(split):
            lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model_split(spectrum_list, labels_list, train_order, 
                                                                     num, num_samples//split*i, num_samples//split*(i+1))

            classifier.fit([lstm_inputs, cnn_inputs], labels_inputs, epochs=1, batch_size=batchsize, verbose=1,
                          validation_data = ([lstm_test, cnn_test], labels_test))
            gc.collect()
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [43]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-24units-model1.hdf5', spec_list, labe_list, epochs=10, times=20)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


Train on 91660 samples, validate on 8340 samples
Epoch 1/10
91660/91660 [==============================] - 159s 2ms/step - loss: 0.0370 - acc: 0.9876 - val_loss: 0.0246 - val_acc: 0.9917
Epoch 2/10
91660/91660 [==============================] - 154s 2ms/step - loss: 0.0229 - acc: 0.9920 - val_loss: 0.0227 - val_acc: 0.9925
Epoch 3/10
91660/91660 [==============================] - 157s 2ms/step - loss: 0.0197 - acc: 0.9930 - val_loss: 0.0246 - val_acc: 0.9923
Epoch 4/10
91660/91660 [==============================] - 154s 2ms/step - loss: 0.0179 - acc: 0.9935 - val_loss: 0.0210 - val_acc: 0.9931
Epoch 5/10
91660/91660 [==============================] - 159s 2ms/step - loss: 0.0167 - acc: 0.9939 - val_loss: 0.0216 - val_acc: 0.9924
Epoch 6/10
91660/91660 [==============================] - 157s 2ms/step - loss: 0.0157 - acc: 0.9941 - val_loss: 0.0198 - val_acc: 0.9931
Epoch 7/10
91660/91660 [==============================] - 157s 2ms/step - loss: 0.0149 - acc: 0.9944 - val_loss: 0.0195 - v

28194

In [44]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_1(optimizer=adam)
train_test_whole(classifier, 0, 'weights/weight-24units-model2.hdf5', spec_list, labe_list, epochs=10, times=20)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':


Train on 91660 samples, validate on 8340 samples
Epoch 1/10
91660/91660 [==============================] - 140s 2ms/step - loss: 0.0377 - acc: 0.9873 - val_loss: 0.0238 - val_acc: 0.9918
Epoch 2/10
91660/91660 [==============================] - 136s 1ms/step - loss: 0.0228 - acc: 0.9920 - val_loss: 0.0201 - val_acc: 0.9932
Epoch 3/10
91660/91660 [==============================] - 135s 1ms/step - loss: 0.0193 - acc: 0.9931 - val_loss: 0.0185 - val_acc: 0.9935
Epoch 4/10
91660/91660 [==============================] - 135s 1ms/step - loss: 0.0171 - acc: 0.9937 - val_loss: 0.0229 - val_acc: 0.9928
Epoch 5/10
91660/91660 [==============================] - 138s 2ms/step - loss: 0.0158 - acc: 0.9941 - val_loss: 0.0180 - val_acc: 0.9940
Epoch 6/10
91660/91660 [==============================] - 138s 2ms/step - loss: 0.0146 - acc: 0.9945 - val_loss: 0.0182 - val_acc: 0.9936
Epoch 7/10
91660/91660 [==============================] - 140s 2ms/step - loss: 0.0137 - acc: 0.9948 - val_loss: 0.0199 - v

23088

In [30]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = classifier_3(optimizer=adam)
train_test_part(classifier, 0, 'weights/weight-24units-model1.hdf5', spec_list, labe_list, epochs=10, split=10)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  



        
        THIS IS EPOCH_0
Train on 4580 samples, validate on 4170 samples
Epoch 1/1
4580/4580 [==============================] - 15s 3ms/step - loss: 0.0994 - acc: 0.9708 - val_loss: 0.0630 - val_acc: 0.9843
Train on 4580 samples, validate on 4170 samples
Epoch 1/1
3072/4580 [===================>..........] - ETA: 2s - loss: 0.0525 - acc: 0.9848

KeyboardInterrupt: 